In [2]:
!pip install torch


   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   -- ------------------------------------- 5.8/109.3 MB 32.9 MB/s eta 0:00:04
   ---- ----------------------------------- 12.8/109.3 MB 32.6 MB/s eta 0:00:03
   ------- -------------------------------- 19.4/109.3 MB 31.8 MB/s eta 0:00:03
   --------- ------------------------------ 26.7/109.3 MB 32.4 MB/s eta 0:00:03
   ------------ --------------------------- 34.6/109.3 MB 33.1 MB/s eta 0:00:03
   --------------- ------------------------ 42.2/109.3 MB 33.4 MB/s eta 0:00:03
   ------------------ --------------------- 49.3/109.3 MB 33.8 MB/s eta 0:00:02
   -------------------- ------------------- 56.9/109.3 MB 34.0 MB/s eta 0:00:02
   ---------------------- ----------------- 61.6/109.3 MB 32.9 MB/s eta 0:00:02
   ------------------------ --------------- 67.9/109.3 MB 32.4 MB/s eta 0:00:02
   --------------------------- ------------ 74.7/109.3 MB 32.3 MB/s eta 0:00:02
   ----------------------------- ---------- 79.4/1

In [3]:
import torch

class Node:
    __slots__ = ("val","next")
    def __init__(self, val:int):
        self.val = val
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self._size = 0
        self._cache = None         
        self._cache_valid = False 

    def _invalidate_cache(self):
        self._cache_valid = False
        self._cache = None

    def is_empty(self):
        return self._size == 0

    def to_list(self):
        out = []
        cur = self.head
        while cur:
            out.append(cur.val)
            cur = cur.next
        return out

    def push_front(self, x:int):
        n = Node(x)
        if self.is_empty():
            self.head = self.tail = n
        else:
            n.next = self.head
            self.head = n
        self._size += 1
        self._invalidate_cache()

    def push_back(self, x:int):
        n = Node(x)
        if self.is_empty():
            self.head = self.tail = n
        else:
            self.tail.next = n
            self.tail = n
        self._size += 1
        self._invalidate_cache()

    def pop_front(self):
        if self.is_empty():
            print("List is empty.")
            return
        v = self.head.val
        self.head = self.head.next
        if self.head is None:
            self.tail = None
        self._size -= 1
        self._invalidate_cache()
        print("Deleted from front:", v)

    def pop_back(self):
        if self.is_empty():
            print("List is empty.")
            return
        if self.head == self.tail:
            v = self.tail.val
            self.head = self.tail = None
            self._size -= 1
            self._invalidate_cache()
            print("Deleted from back:", v)
            return
        # find penultimate
        cur = self.head
        while cur.next != self.tail:
            cur = cur.next
        v = self.tail.val
        cur.next = None
        self.tail = cur
        self._size -= 1
        self._invalidate_cache()
        print("Deleted from back:", v)

    def update_first(self, old_val:int, new_val:int):
        cur = self.head
        while cur:
            if cur.val == old_val:
                cur.val = new_val
                self._invalidate_cache()
                print(f"Updated first occurrence of {old_val} to {new_val}.")
                return
            cur = cur.next
        print(f"Value {old_val} not found.")

    def show(self):
        print("List:", self.to_list(), f"(size={self._size})")

    
    def sort(self):
        def split(head):
            slow, fast = head, head.next
            while fast and fast.next:
                slow = slow.next
                fast = fast.next.next
            mid = slow.next
            slow.next = None
            return head, mid

        def merge(a, b):
            dummy = Node(0)
            tail = dummy
            while a and b:
                if a.val <= b.val:
                    tail.next = a; a = a.next
                else:
                    tail.next = b; b = b.next
                tail = tail.next
            tail.next = a if a else b
            return dummy.next

        def merge_sort(head):
            if head is None or head.next is None:
                return head
            left, right = split(head)
            left  = merge_sort(left)
            right = merge_sort(right)
            return merge(left, right)

        # run merge sort and also fix tail
        self.head = merge_sort(self.head)
        # fix tail
        cur = self.head
        prev = None
        while cur:
            prev = cur
            cur = cur.next
        self.tail = prev

        # update cache (sorted view)
        self._cache = torch.tensor(self.to_list(), dtype=torch.int64)
        self._cache_valid = True
        print("List sorted (ascending).")

   
    def search(self, x:int):
        # ensure we have sorted cache
        if not self._cache_valid:
            # build or rebuild once (sorted view)
            arr = self.to_list()
            arr.sort()
            self._cache = torch.tensor(arr, dtype=torch.int64)
            self._cache_valid = True

        # binary search with torch.searchsorted
        idx = torch.searchsorted(self._cache, torch.tensor([x], dtype=torch.int64), right=False).item()
        found = (idx < len(self._cache)) and (self._cache[idx].item() == x)
        if found:
            print(f"Found {x} at sorted index {idx}.")
        else:
            print(f"{x} not found. Insertion point: {idx}.")
        return found, idx



def get_int(prompt: str):
    while True:
        s = input(prompt).strip()
        try:
            return int(s)
        except ValueError:
            print("Please enter an integer.")

def menu():
    ll = LinkedList()
    actions = {
        "1": lambda: ll.push_front(get_int("Enter INT to insert at FRONT: ")),
        "2": lambda: ll.push_back(get_int("Enter INT to insert at BACK: ")),
        "3": ll.pop_front,
        "4": ll.pop_back,
        "5": lambda: ll.update_first(get_int("OLD value: "), get_int("NEW value: ")),
        "6": ll.show,
        "7": ll.sort,                                  # O(n log n) merge sort
        "8": lambda: ll.search(get_int("Search INT: ")),# O(log n) via tensor
        "0": lambda: (_ for _ in ()).throw(SystemExit),
    }
    while True:
        print("\n--- Linked List Menu ---")
        print("1) Insert front")
        print("2) Insert back")
        print("3) Delete front")
        print("4) Delete back")
        print("5) Update first occurrence")
        print("6) Read (print) list")
        print("7) Sort (ascending)")
        print("8) Search (binary search)")
        print("0) Exit")
        ch = input("Choose: ").strip()
        try:
            if ch in actions: actions[ch]()
            else: print("Invalid choice.")
        except SystemExit:
            print("Goodbye!")
            break
        except Exception as e:
            print("Error:", e)

if __name__ == "__main__":
    menu()



--- Linked List Menu ---
1) Insert front
2) Insert back
3) Delete front
4) Delete back
5) Update first occurrence
6) Read (print) list
7) Sort (ascending)
8) Search (binary search)
0) Exit


: 